# INFO408 Lab Exercise 11: Apache Spark

The aim of this lab exercise is to explore the features of Apache Spark for processing large data sets.

This Jupyter notebook is designed to interact with Apache Spark. It uses Python 3.6.

## A quick reminder of how to use Jupyter notebooks

Type `Control-Enter` in a cell to run the code. If you see `[*]` to the left of a cell being run, the code is still running. (The Spark commands in Exercise 2 analyse half a million email messages, and will therefore take some time to run.)

Use the **Insert** menu to add new cells.

Save your progress using **File** ▷ **Save and Checkpoint**.

When you have finished this lab, you can create a (non-interactive) copy of this notebook in HTML format using **File** ▷ **Download as…**

To exit, select **File** ▷ **Close and Halt**. Do not worry if error messages are displayed in the terminal window from which you started the cluster. You can then type `Control-C` followed by `docker-compose down` to shut down the cluster.

## Useful links

Note that this exercise uses version 2.4.3 of Spark, not version 3.

 * [Spark RDD programming guide](https://spark.apache.org/docs/2.4.3/rdd-programming-guide.html)
 * [Python API documentation for the RDD class](https://spark.apache.org/docs/2.4.3/api/python/pyspark.html#pyspark.RDD)
 * [Spark SQL, DataFrames and Datasets Guide](https://spark.apache.org/docs/2.4.3/sql-programming-guide.html)
 * [Python API documentation for the DataFrame class](https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.DataFrame)
 * [Python standard library (standard datatypes and functions)](https://docs.python.org/3.6/library/index.html)

## Exercise 1: Exploring the Shakespeare data set

In this exercise you will use a data file (`shakespeare.txt`) containing the complete works of William Shakespeare (plays and poems). This comprises about 8.8 MB of semi-structured text.

To query the data using Spark, run the code in each of the following cells by clicking on a cell and then typing `Control-Enter` (`⌘-Enter` on a Mac). Then look at the output, and try to understand what the code is doing.

The variable `sc` is predefined by PySpark to hold a reference to a *Spark Context* object. This is configured to run Spark in the cluster that you created by running `docker-compose up`. Each node in the cluster is available to store parts of the data and to run computations on them. You can also submit jobs to the cluster via `pyspark` (the Python Spark interpreter) running inside this notebook.

In [1]:
sc

<SparkContext master=spark://spark-master:7077 appName=pyspark-shell>

Now load the data.

In [2]:
lines = sc.textFile("/mnt/data/shakespeare.txt")
lines

/mnt/data/shakespeare.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

The variable `lines` now holds a reference to an object of class RDD: a *resilient distributed dataset*. The dataset is automatically partitioned with each partition held in memory in one of the executor processes.

Now let us see how many lines of data were read.

In [3]:
lines.count()

175436

The cell below calls the `take(n)` method. This retrieves the first *n* data items from the RDD and returns them as a Python list, so the list must be small enough to fit into memory.

In [4]:
lines.take(10)

['hamlet@0\t\tHAMLET',
 'hamlet@8\t',
 'hamlet@9\t',
 'hamlet@10\t\tDRAMATIS PERSONAE',
 'hamlet@29\t',
 'hamlet@30\t',
 'hamlet@31\tCLAUDIUS\tking of Denmark. (KING CLAUDIUS:)',
 'hamlet@74\t',
 'hamlet@75\tHAMLET\tson to the late, and nephew to the present king.',
 'hamlet@131\t']

Note: `\t` represents a tab character.

Note that each line begins with a *key*: the name of the work (a play or poem) followed by `@` and a byte offset from the start of the work.

Now let us see how the data has been partitioned by Spark.

In [5]:
lines.getNumPartitions()

2

The result of `getNumPartitions()` shows that the data has been broken up into separate partitions by Spark. The number of partitions would probably be larger if you were running this code on a “real” Spark cluster.

Next, to see the range of methods (functions associated with a class) that are available to be called on an RDD object, position the cursor after the `.` in the cell below and press the `Tab` key (you may need to delete and re-enter the `.` for this to work, and it may be slow to respond). Use the `Page Down` key to see all the methods, and `Esc` to close the list. If you want to read what these methods do, see the [documentation for the DataFrame class](https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.DataFrame).

Note: You do not need to *run* the cell below, but if you do, you will get an error message unless you complete the line to call an RDD class method with valid arguments.

In [6]:
lines.

SyntaxError: invalid syntax (<ipython-input-6-be8527e5aee1>, line 1)

Now let us get rid of the key at the start of each line in the data set. Compare the result with that from `lines.take(10)` above. The `map` method takes a function as an argument and applies it to each item in the data set (in this case, a line of text). A new RDD is returned with each item replaced by its mapped value.

Here we do the mapping using a lambda function (see Lab 4). This takes one string argument (`line`), splits it at the first (`maxsplit=1`) tab character (`sep="\t"`) and returns the second substring (`[1]`, because Python indexes lists from 0). As you can see from the result this is not perfect, as there are still some lines that start with a tab character, but it is good enough for our purposes. (If you are interested, investigate Python’s `re` module that provides advanced pattern matching and substitution using regular expressions.)

In [7]:
import re
lines2 = lines.map(lambda line: line.split(sep="\t", maxsplit=1)[1])
lines2.take(10)

['\tHAMLET',
 '',
 '',
 '\tDRAMATIS PERSONAE',
 '',
 '',
 'CLAUDIUS\tking of Denmark. (KING CLAUDIUS:)',
 '',
 'HAMLET\tson to the late, and nephew to the present king.',
 '']

Python’s `lambda` syntax is very useful for passing short unnamed functions to functions like map and reduce. The general syntax is:

```python
  lambda arg1, ..., argn: SomeExpressionUsingTheArguments
```

The expression is evaluated and its result is returned when the function is called.

Now we will create a new RDD containing the description of the plays’ scenes. The `collect` method retrieves all data from an RDD and returns it as a Python list. You should therefore `collect` if the RDD is small enough to fit into memory.

In [8]:
scenes = lines2.filter(lambda line: "SCENE " in line)
scenes.collect()

['SCENE I\tElsinore. A platform before the castle.',
 'SCENE II\tA room of state in the castle.',
 "SCENE III\tA room in Polonius' house.",
 'SCENE IV\tThe platform.',
 'SCENE V\tAnother part of the platform.',
 "SCENE I\tA room in POLONIUS' house.",
 'SCENE II\tA room in the castle.',
 'SCENE I\tA room in the castle.',
 'SCENE II\tA hall in the castle.',
 'SCENE III\tA room in the castle.',
 "SCENE IV\tThe Queen's closet.",
 'SCENE I\tA room in the castle.',
 'SCENE II\tAnother room in the castle.',
 'SCENE III\tAnother room in the castle.',
 'SCENE IV\tA plain in Denmark.',
 'SCENE V\tElsinore. A room in the castle.',
 'SCENE VI\tAnother room in the castle.',
 'SCENE VII\tAnother room in the castle.',
 'SCENE I\tA churchyard.',
 'SCENE II\tA hall in the castle.',
 'SCENE I\tLondon. A street.',
 'SCENE II\tThe same. Another street.',
 'SCENE III\tThe palace.',
 'SCENE IV\tLondon. The Tower.',
 'SCENE I\tLondon. The palace.',
 'SCENE II\tThe palace.',
 'SCENE III\tLondon. A street.',
 

Next, we generate a sorted RDD of pairs (count, word) for each word in the data set. `sortByKey(ascending=False)` means that the result will be sorted in *descending* order, i.e., most frequently used words first. If you want to see the *least* frequently used words, change `False` to `True`.

In [9]:
punctuation = ".?,:;-"
remove_punct_mapping = dict.fromkeys(map(ord, punctuation)) # Do not worry about what this means
wordcounts = \
    lines2.map(lambda x: x.translate(remove_punct_mapping).lower()) \
    .flatMap(lambda x: x.split()) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda x,y: x+y) \
    .map(lambda x: (x[1],x[0])) \
    .sortByKey(ascending=False)
wordcounts.take(10)

[(30011, 'the'),
 (27496, 'and'),
 (20960, 'i'),
 (20703, 'to'),
 (18764, 'of'),
 (15800, 'a'),
 (13876, 'you'),
 (12926, 'my'),
 (11761, 'in'),
 (11496, 'that')]

The backslash characters at the end of some of the lines above tell Python that the current line of code continues on the next line. This is useful when we have to chain together a long series of method calls.

An alternative approach to these long call chains would be to assign the result of each method call to a new variable, and progressively call the methods on each intermediate RDD.

By the way, Spark uses a “lazy” approach to generating RDDs, similar to PETL (Lab 4). It usually does no computation on data (including the generation of intermediate RDDs) until some results are actually requested, e.g. by calling `take` or `collect`.

To understand how the code above works, you can try a simpler example from http://www.mccarroll.net/blog/pyspark2/ (“Revisiting the wordcount example”). We have reproduced the code from that web page below, starting from the second shaded box onwards (note that the code for removing punctuation is different from the improved version above). Read the description of each step and then try running it.

**Code from the second shaded box:**

In [10]:
lines = sc.parallelize(['Its fun to have fun,','but you have to know how.']) 
wordcounts = lines.map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower()) \
    .flatMap(lambda x: x.split()) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda x,y:x+y) \
    .map(lambda x:(x[1],x[0])) \
    .sortByKey(False) 
wordcounts.take(10)

[(2, 'have'),
 (2, 'fun'),
 (2, 'to'),
 (1, 'its'),
 (1, 'you'),
 (1, 'how'),
 (1, 'but'),
 (1, 'know')]

**Code for step 1: `map(<function>)`**

In [11]:
r1 = lines.map( lambda x: x.replace(',',' ').replace('.',' ').replace('-',' ').lower())
r1.take(10)

['its fun to have fun ', 'but you have to know how ']

**Code for step 2: `flatMap(<function>)`**

In [12]:
r2 = r1.flatMap(lambda x: x.split())
r2.take(20)

['its', 'fun', 'to', 'have', 'fun', 'but', 'you', 'have', 'to', 'know', 'how']

**Code for step 3: `map(<function>)` (again)**

In [13]:
r3 = r2.map(lambda x: (x, 1))
r3.take(20)

[('its', 1),
 ('fun', 1),
 ('to', 1),
 ('have', 1),
 ('fun', 1),
 ('but', 1),
 ('you', 1),
 ('have', 1),
 ('to', 1),
 ('know', 1),
 ('how', 1)]

**Code for step 4: `reduceByKey(<function>)`**

In [14]:
r4 = r3.reduceByKey(lambda x,y:x+y)
r4.take(20)

[('have', 2),
 ('but', 1),
 ('know', 1),
 ('its', 1),
 ('fun', 2),
 ('to', 2),
 ('you', 1),
 ('how', 1)]

**Code for step 5: `map(<function>)` (yet again!)**

In [15]:
r5 = r4.map(lambda x:(x[1],x[0]))
r5.take(20)

[(1, 'but'),
 (2, 'have'),
 (1, 'know'),
 (1, 'you'),
 (2, 'to'),
 (1, 'how'),
 (1, 'its'),
 (2, 'fun')]

**Code for step 6: `sortByKey( ascending=True|False )`**

In [16]:
r6 = r5.sortByKey(ascending=False)
r6.take(20)

[(2, 'have'),
 (2, 'to'),
 (2, 'fun'),
 (1, 'but'),
 (1, 'know'),
 (1, 'you'),
 (1, 'how'),
 (1, 'its')]

Now, leaving that example, make sure that you understand the difference between the functions `map` and `flatMap`, by predicting what each of the following three cells will do, and then running them to check whether your prediction is correct.

In [17]:
words = sc.parallelize(["aardvark", "baby", "carrot"]) # Create an RDD from a Python list
word_chars = words.map(list)  # The function 'list' converts a string to a list of characters (which are just short strings)
word_chars.collect() # Collect gathers the RDD into a Python list.

[['a', 'a', 'r', 'd', 'v', 'a', 'r', 'k'],
 ['b', 'a', 'b', 'y'],
 ['c', 'a', 'r', 'r', 'o', 't']]

In [18]:
all_chars = words.flatMap(list)
all_chars.collect()

['a',
 'a',
 'r',
 'd',
 'v',
 'a',
 'r',
 'k',
 'b',
 'a',
 'b',
 'y',
 'c',
 'a',
 'r',
 'r',
 'o',
 't']

In [19]:
distinct_chars = all_chars.distinct()
distinct_chars.collect()

['r', 'd', 'b', 'y', 'c', 'a', 'o', 't', 'v', 'k']

## Exercise 2: Exploring the Enron dataset

Next, you will explore a dataset of email messages from the Enron corporation. You can read about it [here](https://en.wikipedia.org/wiki/Enron_Corpus).

First, remove the Exercise 1 dataset from memory. The output should be 0, indicating the success of the garbage collection.

In [20]:
%reset -sf
import gc
gc.collect()

0

Rather than reading the data into a Spark resilient distributed dataset (RDD), we will  create a Spark *data frame* from the JSON data file. A data frame includes schema information (inferred from the JSON file in this case).

The `printSchema()` method call below prints the schema, and the `first()` method call prints the first message in the data set. The variable `spark` is predefined by PySpark to hold a reference to a *Spark Session* object, which is used instead of the Spark context to create a data frame.

This code will probably while take a while to run.

In [21]:
messages = spark.read.json('/mnt/data/messages.json')
messages.createOrReplaceTempView("messages") # This is needed before doing any subsequent SQL queries on this data frame
print(messages, '\n')
messages.printSchema() # Print the schema
print('Number of messages:', messages.count())
messages.first() # Look at the first email message

DataFrame[_id: struct<$oid:string>, body: string, filename: string, headers: struct<Attendees:string,Bcc:string,Cc:string,Content-Transfer-Encoding:string,Content-Type:string,Date:string,From:string,Message-ID:string,Mime-Version:string,Re:string,Subject:string,Time:string,To:string,X-FileName:string,X-Folder:string,X-From:string,X-Origin:string,X-To:string,X-bcc:string,X-cc:string>, mailbox: string, subFolder: string] 

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- body: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- headers: struct (nullable = true)
 |    |-- Attendees: string (nullable = true)
 |    |-- Bcc: string (nullable = true)
 |    |-- Cc: string (nullable = true)
 |    |-- Content-Transfer-Encoding: string (nullable = true)
 |    |-- Content-Type: string (nullable = true)
 |    |-- Date: string (nullable = true)
 |    |-- From: string (nullable = true)
 |    |-- Message-ID: string (nullable = true)
 |    |-- Mime

Row(_id=Row($oid='4f16fc97d1e2d32371003e27'), body="the scrimmage is still up in the air...\n\n\nwebb said that they didnt want to scrimmage...\n\nthe aggies  are scrimmaging each other... (the aggie teams practiced on \nSunday)\n\nwhen I called the aggie captains to see if we could use their field.... they \nsaid that it was tooo smalll for us to use...\n\n\nsounds like bullshit to me... but what can we do....\n\n\nanyway... we will have to do another practice Wed. night....    and I dont' \nknow where we can practice.... any suggestions...\n\n\nalso,  we still need one  more person...", filename='450.', headers=Row(Attendees=None, Bcc=None, Cc=None, Content-Transfer-Encoding='7bit', Content-Type='text/plain; charset=us-ascii', Date='Tue, 14 Nov 2000 08:22:00 -0800 (PST)', From='michael.simmons@enron.com', Message-ID='<6884142.1075854677416.JavaMail.evans@thyme>', Mime-Version='1.0', Re=None, Subject='Re: Plays and other information', Time=None, To='eric.bass@enron.com', X-FileName='e

Spark data frames support the use of SQL (extended to allow nested column/property names, e.g. `headers.From`).

In [22]:
# Find the total number of message senders in the data set
spark.sql("select count(distinct headers.From) fromcount from messages").collect()

[Row(fromcount=19966)]

The result is expressed as a DataFrame that comprises a single row.

There are also a range of useful methods we can use on our data frame, for example:

In [23]:
# Count emails from louis.soldano@...
messages.filter(messages.headers.From == 'louis.soldano@enron.com').count()

105

## Assessment

Answer the questions below by writing your Python code in the corresponding cells below. Save the modified notebook and upload it to Blackboard. You do not need to provide the output from the code. The deadline to submit your answers is **Thursday 1 October at 5pm.**

Explore the use of SQL queries and/or dataframe and RDD methods to summarise and extract information from the Enron dataset. The following web page might be useful to see what operations can be performed on data frames and RDDs: https://github.com/lgallen/pyspark_dataframes/blob/master/housing_model.ipynb (section “Dataframe operations”).

Documentation for the methods that can be called on data frames: https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.DataFrame

Documentation for the methods that can be called on RDDs: http://spark.apache.org/docs/2.4.3/api/python/pyspark.html#pyspark.RDD

Some built-in Spark functions: http://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#module-pyspark.sql.functions

Once you have tried some basic queries, try to answer the questions below. It is OK to report on partially successful attempts! You can add commentary in code comments or additional text cells. Also feel free to invent your own questions about this dataset to answer using Spark.

<ol>
    <li>
        <p><strong>What is the distribution of message body word counts across all messages?</strong> In other words, create an RDD or data frame that maps word counts (how many words are in a message body) to the number of messages with that word count. Rather than giving exact word counts, round them to the closest power of 10 (you can use <code>round(count, -1)</code>). Note that <code>messages</code> is a data frame, whereas this problem might be easily solved using the methods that can be called on an RDD. However, you can extract an RDD from the data frame, e.g:</p>
    </li>
</ol>

In [24]:
messagesRdd = messages.rdd

<ol>
    <li style="list-style: none;">
        <p>The new RDD will contain <code>Row</code> objects. If you call <code>map()</code> on <code>messagesRdd</code> as defined above, the function you provide to <code>map</code> will need to take a row object as its parameter, e.g. <code>messagesRdd.map(lambda row: ...)</code>. See the <a href="https://spark.apache.org/docs/2.4.3/api/python/pyspark.sql.html#pyspark.sql.Row" target="_blank">Row class documentation</a> to find out how to access the columns of a row (basically, it is the same as for a data frame).</p>
    </li>
</ol>

In [25]:
# Code to answer question 1.
split_messages = messagesRdd.map(lambda row: row.body.split())

shakespeare_count=split_messages.map(lambda  word:(word))

shakespeare_count_RBK=shakespeare_count.map(lambda x: round(len(x),-1))

final = shakespeare_count_RBK.map(lambda x: (x, 1)) \
        .reduceByKey(lambda x,y:x+y)

final2 = final.sortByKey(True)
final.take(4)


[(60, 23450), (0, 12995), (480, 1805), (180, 9967)]

<ol start="2">
    <li>
        <p><strong>Which words appear most commonly within message subject lines?</strong></p>
        <p><strong>Hint:</strong> this is very similar to the standard map-reduce example of counting words in documents — see the word count example in Exercise 1. The data processed in that example is an RDD, whereas we have a data frame, so you may want to use <code>messagesRdd</code> as defined above.</p>
        <p><strong>Tip for sorting an RDD:</strong> To sort an RDD containing key-value pairs (represented in Python as <code>(key, value)</code> tuples), you can use the RDD method <code>sortBy()</code>. To sort by <em>value</em> in descending order, call <code>sortBy(lambda pair: pair[1], ascending=False)</code>.</p>
    </li>
</ol>

In [26]:
# Code to answer question 2.
punctuation = ".?,:;-"
remove_punct_mapping = dict.fromkeys(map(ord, punctuation)) # Do not worry about what this means
wordcounts = \
    messagesRdd.map(lambda x: x.headers.Subject) \
    .map(lambda x: x.translate(remove_punct_mapping).lower()) \
    .flatMap(lambda x: x.split()) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda x,y: x+y) \
    .sortBy(lambda pair: pair[1], ascending=False)
    
wordcounts.first()


('re', 151321)

<ol start="3">
    <li><strong>Which senders sent the most messages?</strong></li>
</ol>

In [27]:
# Code to answer question 3.
senderCounts = \
    messagesRdd.map(lambda x: x.headers.From) \
    .map(lambda x: (x, 1)) \
    .reduceByKey(lambda x,y: x+y) \
    .max(lambda x: x[1]) \
    
senderCounts


('kay.mann@enron.com', 16732)

## Challenge question (not assessed)

<p><strong>What are the unique sender/recipient pairs together with the number of times each pair occurs?</strong></p>
<p><strong>Note:</strong> This query requires parsing the <code>headers.To</code> string to find the individual recipients, as in general an email message may have more than one recipient, separated by commas followed by a space, tab or newline character. Extracting individual recipients can be done using map-reduce on the RDD underlying the data frame. We can also do this using the <a href="https://docs.databricks.com/spark/latest/spark-sql/language-manual/select.html#lateral-view" target="_blank">Spark SQL <em>lateral view</em> construct</a> and the <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.explode" target="_blank"><code>explode</code></a> and <a href="https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.split" target="_blank"><code>split</code></a> functions.</p>
<p>The following query creates a new data frame with columns named <code>sender</code> and <code>recipient</code>. There is a row for each occurrence of a sender-recipient pair in the data set (there may be multiple entries for any given pair). (Note that the column names <code>from</code> and <code>to</code> have been changed to <code>sender</code> and <code>recipient</code> to avoid confusing Spark SQL, which does not like a column named <code>from</code>.)</p>

In [ ]:
sender_recipient_pairs = spark.sql("select messages.headers.From as sender, recipient, count(*) as count from messages lateral view explode(split(messages.headers.To, '[,\\\\s]+')) tolist as recipient group by sender, recipient order by count desc")
sender_recipient_pairs.createOrReplaceTempView("sender_recipient_pairs")
sender_recipient_pairs.printSchema()
sender_recipient_pairs.take(5)

root
 |-- sender: string (nullable = true)
 |-- recipient: string (nullable = true)
 |-- count: long (nullable = false)



The next cell produces a new data frame that excludes the emails sent or CCed to self.

In [ ]:
no_self_mails_sender_recipient_pairs = spark.sql("select sender, recipient, count from sender_recipient_pairs where sender <> recipient")
no_self_mails_sender_recipient_pairs.createOrReplaceTempView("no_self_mails_sender_recipient_pairs")
no_self_mails_sender_recipient_pairs.take(5)

We have just shown you how to answer this question using data frames, so try to answer this question using a different technique (e.g., convert the dataframe to an RDD and use RDD methods).

In [ ]:
# Code to answer question 4.
